In [1]:
import torch

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\torchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(1)

# 利用Embedding的方式來建立N-gram模型

In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [4]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [5]:
ngrams = [([test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))]

In [6]:
print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [7]:
vocab = set(test_sentence)

In [8]:
word_to_ix = {word : i for i, word in enumerate(vocab)}

In [9]:

class NGramLanguageModeler(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size) -> None:
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = torch.nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = torch.nn.Linear(128, vocab_size)

    def forward(self, x):
        x = self.embeddings(x).view((1, -1)) # 這邊的想法是concat兩個tensor的向量後統合一個結果
        x = torch.nn.functional.relu(self.linear1(x))
        x = self.linear2(x)
        x = torch.nn.functional.log_softmax(x, dim=1)
        return x

In [10]:
losses = []
loss_function = torch.nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[519.948404788971, 517.5656819343567, 515.1981420516968, 512.8428540229797, 510.5008637905121, 508.17082953453064, 505.85368847846985, 503.54755997657776, 501.2524297237396, 498.96888995170593]
tensor([-1.4242, -1.3243, -2.0825, -0.6312,  1.5993,  1.9345,  0.5939,  1.8199,
        -0.8790, -1.1806], grad_fn=<SelectBackward0>)
